# visualize surprisal development


In [1]:
from pathlib import Path
import pandas as pd

ROOT = Path("/Users/jliu/workspace/RAG/")
surprisal_path = ROOT / "results" / "surprisal"

In [8]:
def format_csv(file_path:Path)->pd.DataFrame:
    """Remove the target col from the csv file."""
    df = pd.read_csv(file_path)
    #df = df.drop('ablated_neurons', axis=1)
    # convert differnet steps into different rows
    df_h = df[df["step"]==0]
    df_h = df_h.drop('surprisal', axis=1)
    df_grouped = df.groupby("step")
    for step, df_group, in df_grouped:
        df_h[step] = df_group['surprisal'].to_list()
    return df_h

In [9]:
# load freq file
file_path = surprisal_path/"pythia-example.csv"
df_h = format_csv(file_path)